# 依存関係分析プログラム

# **ライブラリのインポート（最初に実行）**

In [1]:
import spacy
import os
from spacy.tokens import DocBin
from spacy.matcher import DependencyMatcher
from spacy import displacy

# **解析データの読み込み**
## ユーザーが設定するパラメーター

In [2]:
# 基本設定
main_directory = "processed_data"  # 親ディレクトリへの相対パス
subdirectories = ["sub1"]  # 読み込む子ディレクトリ名のリスト
nlp_model = "en_core_web_sm"  # 解析に利用したモデル名

# 特定のファイルのみを読み込む場合は以下のリストにtxtファイル名を追加(空リストの場合は全ファイルを読み込む)
selected_file_names = []

## 解析データ読み込みの実行

In [ ]:
# spaCyの言語モデルをロード
nlp = spacy.load(nlp_model)

# 解析結果を格納する辞書の初期化
docs_dict = {}
# フィルタリングされたDocオブジェクトを格納する辞書の初期化
filtered_docs = {}

# 指定されたサブディレクトリ内のファイルを読み込む
for subdir in subdirectories:
    directory = os.path.join(main_directory, subdir)
    
    # サブディレクトリ内のファイルを読み込む
    for filename in os.listdir(directory): 
        if filename.endswith(".spacy"):  # .spacyファイルならば
            spacy_path = os.path.join(directory, filename)
            txt_filename = filename.replace(".spacy", ".txt")  # .spaCyファイルに対応するテキストファイル名を作成
            txt_path = os.path.join(directory, txt_filename)  # .spacyファイルに対応するテキストファイルのパス
            
            # .spacyファイルからDocオブジェクトを読み込む
            doc_bin = DocBin().from_disk(spacy_path)  # DocBinオブジェクトを読み込む
            docs = list(doc_bin.get_docs(nlp.vocab))  # Docオブジェクトをリストに格納
            
            # 対応する.txtファイルからファイル名を読み込む
            if os.path.exists(txt_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    file_names = [line.strip() for line in f.readlines()]  # ファイル名をリストに格納
                
                # Docオブジェクトとファイル名の対応を辞書に格納
                for doc, fname in zip(docs, file_names):
                    docs_dict[fname] = doc
            else:
                print(f"警告: {txt_filename} に対応するテキストファイルが存在しません。")

# ユーザーがファイル名を指定した場合、それに対応するDocオブジェクトをフィルタリング
if selected_file_names:
    missing_files = []  # 存在しないファイル名を格納するリスト
    for fname in selected_file_names:
        if fname in docs_dict:  # 指定されたファイル名が読み込んだデータに存在する場合
            filtered_docs[fname] = docs_dict[fname]
        else:
            missing_files.append(fname)
    
    if missing_files:
        print(f"警告: 次の指定されたファイルは存在しません: {', '.join(missing_files)}")
    if filtered_docs:
        print("指定された一部のデータが読み込まれました。")
    else:
        print("指定されたファイルに対応するデータが見つかりませんでした。")
else:
    filtered_docs = docs_dict
    print("全てのデータが読み込まれました。")
print(f"読み込まれたデータ数: {len(filtered_docs)}")
print(f"読み込まれたファイル名: {list(filtered_docs.keys())}")

# **依存関係マッチャ―**
## ユーザーが設定するパラメーター

In [23]:
pattern = [
    {"RIGHT_ID": "verb", "RIGHT_ATTRS": {"POS": "VERB"}},
    {"LEFT_ID": "verb", "REL_OP": ">", "RIGHT_ID": "subject", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": ["bank", "fed", "government"]}}},
    {"LEFT_ID": "verb", "REL_OP": ">", "RIGHT_ID": "object", "RIGHT_ATTRS": {"DEP": "dobj", "LOWER": {"IN": ["rates", "policy", "markets"]}}}
]

# 出力するファイル名
output_file = "dep_matched_sentences.txt"

## Dependency Matcherの実行

In [ ]:
# Dependency Matcherのインスタンスを作成
matcher = DependencyMatcher(nlp.vocab)
# パターンをMatcherに追加
matcher.add("PATTERN", [pattern])

# マッチした文を格納するためのセット（重複を避ける）
matched_sentences_set = set()

# マッチした文を探す
for doc in filtered_docs.values():
    matches = matcher(doc)
    for match_id, token_ids in matches:
        for token_id in token_ids:
            sentence = doc[token_id].sent
            # 文をセットに追加（重複を避ける）
            matched_sentences_set.add(sentence.text)

# マッチした文をテキストファイルに出力
def save_matched_sentences(matched_sentences_set, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        for sentence in matched_sentences_set:
            f.write(f"{sentence}\n")

# マッチした文の総数をプリント
if matched_sentences_set:
    print(f"検索結果: {len(matched_sentences_set)} 件の結果が見つかりました。")
    save_matched_sentences(matched_sentences_set, output_file)
    print(f"検索結果を {output_file} に保存しました。")
else:
    print("検索結果が見つかりませんでした。")

# **依存関係指定の用例抽出**
## ユーザーが設定するパラメーター

In [ ]:
# 抽出する特徴語と依存関係のタイプを指定
feature_words = ["economy", "inflation", "growth", "market", "stock"]
dependency_types = ['dobj', 'pobj', 'nsubj', 'xcomp', 'nsubjpass', 'agent', 'acomp', 'compound', 'prep']

# 出力ディレクトリの設定
output_directory = "dependency_output"

## 用例抽出の実行

In [ ]:
def extract_sentences(docs, feature_words, output_dir, dependency_types):
    """
    指定された依存関係を持つ文を抽出し、特徴語ごとにテキストファイルに出力する関数。

    Args:
        docs (dict): Docオブジェクトを格納した辞書。キーはファイル名、値はDocオブジェクト。
        feature_words (list): 特徴語のリスト。
        output_dir (str): 出力先のディレクトリ名。
        dependency_types (list): 抽出する依存関係のリスト。

    Returns:
        None
    """
    os.makedirs(output_dir, exist_ok=True)
    for word in feature_words:
        output_path = os.path.join(output_dir, f"{word}.txt")
        with open(output_path, 'w', encoding='utf-8') as f:
            for dep_type in dependency_types:
                f.write(f"● {word.upper()} ({dep_type.upper()})\n")
                sentence_count = 1
                for doc_name, doc in docs.items():
                    for sent in doc.sents:
                        for token in sent:
                            if token.lemma_.lower() == word.lower() and token.dep_ == dep_type:
                                f.write(f" {sentence_count}. {sent.text}\n")
                                sentence_count += 1
                f.write("\n")

# 文の抽出と出力
extract_sentences(filtered_docs, feature_words, output_directory, dependency_types)
# 出力の完了を報告
print(f"文の抽出が完了しました。出力先: {output_directory}")
print(f"抽出された特徴語: {feature_words}")

# **依存関係の可視化**
## 依存関係を表示（入力文を解析）

In [ ]:
# 解析したい文を入力
text = "Apple is looking at buying U.K. startup for $1 billion."

# 文をDocオブジェクトに変換
nlp = spacy.load(nlp_model)
doc = nlp(text)
options = {"compact": True, "bg": "#09a3d5","color": "white", "font": "Source Sans Pro", "distance": 110}

# 依存関係を表示
displacy.render(doc, style="dep", jupyter=True, options=options)

## 依存関係を表示（元の解析済みデータから依存関係を取得し表示）

In [ ]:
# 検索キーワードを定義（このキーワードを含む文を検索します）
search_keyword = "Apple is looking at buying U.K. startup for $1 billion"

# マッチした文の依存関係を表示する関数
def display_dependency(doc):
    options = {"compact": True, "bg": "#09a3d5","color": "white", "font": "Source Sans Pro", "distance": 110}
    displacy.render(doc, style="dep", jupyter=True, options=options)

# マッチした文を探す
for file_name, doc in filtered_docs.items():
    # doc内の各文（sent）に対してループ
    for sent in doc.sents:
        if search_keyword in sent.text:
            print(f"ファイル '{file_name}' から見つかった文: {sent.text}")
            display_dependency(sent)
            break  # 1つ見つかった時点でループを抜ける
    # 見つからなかった場合の処理
    else:
        print(f"ファイル '{file_name}' に '{search_keyword}' を含む文は見つかりませんでした。")